# **Quantifier Depth(QD) in FOL**

## **1. Introduction**

This document is written in **RASP** (Restricted Access Sequence Processing), a domain-specific language introduced in *Thinking Like Transformers (Weiss et al., 2021)*, which approximates the self-attention mechanism in the Transformer model. 

The algorithms in this document focus on computing Quantifier Depth (QD) in First-order Logic (FOL) expressions. Syntactically, QD represents the maximum nesting level of quantifiers (e.g., ∀, ∃) in a formula, and is widely used to measure structural logical complexity.

A total of **four FOL types** are covered, each corresponding to a distinct logical structure with varying syntactic depth. The types are ordered with increasing complexity, based on their nesting behavior and compositional logic.

For each **Type**:

- **A BNF grammar** is provided;

- **A RASP algorithm** implements QD computation for instances conforming to that grammar;

- **Several Test Cases** includes "(x/x)" annotated examples demonstrating both the algorithm’s expected behavior and its generalization. The final block in this section follows this standardized pattern:

## **2. Setup**

In [1]:
examples off

### 1-1 Basic Functions in RASP

### 1-2 Symbols and Positions

In [2]:
# Basic lists and s-ops for computinng QD
quanti = ["∀", "∃"];     
LO = ["∧", "∨", "⊕", "→"];
is_quanti = tokens in quanti;
is_LO = tokens in LO;

     list: quanti = ['∀', '∃']
     list: LO = ['∧', '∨', '⊕', '→']
     s-op: is_quanti
     s-op: is_LO


In [3]:
# Boolean value for identifing opening and close brackets
opens = (tokens == "(");
closes = (tokens == ")");

     s-op: opens
     s-op: closes


In [4]:
## Select the position according to the indices
select_all = select(1, 1, ==);
leftward = select(indices, indices, <=);  
equal = select(indices, indices, ==);

## Two-way selecttion of the position of quantifier and LO 
select_quanti = select(is_quanti, True, ==) and select(True, is_quanti, ==);
select_LO = select(is_LO, True, ==) and select(True, is_LO, ==);

     selector: select_all
     selector: leftward
     selector: equal
     selector: select_quanti
     selector: select_LO


### 1-3 The Algorithm for Layered Depth

In [5]:
def num_prevs(bools){
    # Count the number of a boolean condition
    # Reuse from the "num_prev" function in the original paper

    ## 1. Sum and average the number of the boolean condition
    num = aggregate(leftward, indicator(bools)); 

    ## 2. Undo the average value 
    return (indices+1)*num;  
}

     console function: num_prevs(bools)


In [6]:
def split_layers(){
    # Split the FOL by the nesting depth of brackets 

    ## 1. Identify and count the brackets
    n_opens = num_prevs(opens);
    n_closes = num_prevs(closes);
    
    ## 2. Compute nested level at each token
    depth = n_opens - n_closes;

    ## 3. Include close brackets into same depth
    adjusted_depth = depth + indicator(closes);

    return adjusted_depth;
}

     console function: split_layers()


In [7]:
# Assign a same number to tokens in the same depth in order
## e.g. 000111
adjusted_depth = split_layers();

# Select tokens in the same depth
same_depth = select(adjusted_depth, adjusted_depth, ==);

# Compute relative postion within the current depth range
## e.g. 123123
relative_pos = selector_width(same_depth and leftward);

     s-op: adjusted_depth
     selector: same_depth
     s-op: relative_pos


## **3. QD in FOL formulas**

In [8]:
examples on

### **Standard-1**: Simple FOL with no LO

#### Standard-1: Algorithm

In [9]:
def QD_type1_fn(){
    # Count the quantifiers by frequency

    num_quanti = selector_width(select_quanti and leftward);
    ## store the max QD in the last idx
    QD_type1 = num_quanti[-1];

    return QD_type1;
}

     console function: QD_type1_fn()


##### Simple version

In [10]:
def simplify_type1(){
    type1 = aggregate(leftward, indicator(is_quanti))*(indices+1);
    return type1[-1];
}

     console function: simplify_type1()


#### Standard-1: Test Cases (3/3)

In [11]:
type1_1 = "∀uV(u)"; # QD=1
type1_2 = "∃x∀yP(x,y)"; # QD=2
type1_3 = "∀q∃d∃gF(q,d,g)"; # QD=3

    value: type1_1 =  "∀uV(u)"
    value: type1_2 =  "∃x∀yP(x,y)"
    value: type1_3 =  "∀q∃d∃gF(q,d,g)"


In [12]:
set example type1_1
QD_type1 = simplify_type1();
# draw(simplify_type1(), "∀uV(u)");

     s-op: QD_type1
 	 Example: QD_type1("∀uV(u)") = [1.0]*6 (floats)


In [13]:
set example type1_2
QD_type1_2 = simplify_type1();
# draw(simplify_type1(), "∃x∀yP(x,y)");

     s-op: QD_type1_2
 	 Example: QD_type1_2("∃x∀yP(x,y)") = [2.0]*10 (floats)


In [14]:
set example type1_3
QD_type1_3 = simplify_type1();
# draw(simplify_type1(), "∀q∃d∃gF(q,d,g)");

     s-op: QD_type1_3
 	 Example: QD_type1_3("∀q∃d∃gF(q,d,g)") = [3.0]*14 (floats)


### **Standard-2**: Simple FOL with one LO
- It can also process **Type1 QD**.

#### Standard-2: Algorithm

In [15]:
def QD_type2_fn(){
    # Compare the max QD of two clauses

    ## 1. Find the positions of quantifier and LO
    clause_quanti = select(is_quanti, True, ==);
    LO_pos = aggregate(select_LO, indices);
    LO_idx = aggregate(select_all, LO_pos)*length;
    
    ## 2. Extact tokens on both sides of LO and compute their respective quantifier
    left_quant  = clause_quanti and select(indices, LO_idx, <);
    right_quant = clause_quanti and select(indices, LO_idx, >=);
    left_QD  = selector_width(left_quant);
    right_QD = selector_width(right_quant);

    ## 3. Take the side with a deeper QD
    ### It can also be realized by the "max()" function
    QD_type2 = left_QD if (left_QD > right_QD) else right_QD;

    return QD_type2;
}

     console function: QD_type2_fn()


#### Standard-2: Test Cases (5/5)

In [22]:
type2_0 = "∃x∀f∀aQ(xfa)"; # QD=3

type2_1 = "∀pQ(p)∧∀fQ(f)"; # QD=1
type2_2 = "∀v∀hK(h,v)→∀k∀sV(s,k)"; # QD=2
type2_3 = "∀x∀hR(h,x)∨∃f∀qW(q,f)"; # QD=2
type2_4 = "∀xP(x)∨∀y∃z∀wQ(y,z,w)"; # QD=3
type2_5 = "∀gE(g)→∀bF(b)"; # QD=1

    value: type2_0 =  "∃x∀f∀aQ(xfa)"
    value: type2_1 =  "∀pQ(p)∧∀fQ(f)"
    value: type2_2 =  "∀v∀hK(h,v)→∀k∀sV(s,k)"
    value: type2_3 =  "∀x∀hR(h,x)∨∃f∀qW(q,f)"
    value: type2_4 =  "∀xP(x)∨∀y∃z∀wQ(y,z,w)"
    value: type2_5 =  "∀gE(g)→∀bF(b)"


In [21]:
set example type2_0
QD_type2_0 = QD_type2_fn();
draw(QD_type2_fn(), "∃x∀f∀aQ(xfa)");

     s-op: QD_type2_0
 	 Example: QD_type2_0("∃x∀f∀aQ(xfa)") = [3]*12 (ints)
	 =  [3]*12 (ints)


In [147]:
set example type2_1
QD_type2_1 = QD_type2_fn();
# draw(QD_type2_fn(), "∀pQ(p)∧∀fQ(f)");

     s-op: QD_type2_1
 	 Example: QD_type2_1("∀pQ(p)∧∀fQ(f)") = [1]*13 (ints)


In [148]:
set example type2_2
QD_type2_2 = QD_type2_fn();
# draw(QD_type2_fn(), "∀v∀h¬K(h,v)→∀k∀sV(s,k)");

     s-op: QD_type2_2
 	 Example: QD_type2_2("∀v∀hK(h,v)→∀k∀sV(s,k)") = [2]*21 (ints)


In [149]:
set example type2_3
QD_type2_3 = QD_type2_fn();
# draw(QD_type2_fn(), "∀x∀hR(h,x)∨∃f∀q¬W(q,f)");

     s-op: QD_type2_3
 	 Example: QD_type2_3("∀x∀hR(h,x)∨∃f∀qW(q,f)") = [2]*21 (ints)


In [152]:
set example type2_4
QD_type2_4 = QD_type2_fn();
# draw(QD_type2_fn(), "∀xP(x)∨∀y∃z∀wQ(y,z,w)");

     s-op: QD_type2_4
 	 Example: QD_type2_4("∀xP(x)∨∀y∃z∀wQ(y,z,w)") = [3]*21 (ints)
	 =  [3]*21 (ints)


In [23]:
set example type2_5
QD_type2_5 = QD_type2_fn();
# draw(QD_type2_fn(), "∀gE(g)→∀bF(b)");

     s-op: QD_type2_5
 	 Example: QD_type2_5("∀gE(g)→∀bF(b)") = [1]*13 (ints)
	 =  [1]*13 (ints)


### **Standard-3**: Simple QD allowing unlimited LOs
- It can also process **Standard-2 QD**.

#### Standard-3: Algorithm

In [153]:
def QD_type3_fn(){
    clause_idx = aggregate(leftward, indicator(is_LO))*(indices+1);
    same_clause = select(clause_idx, clause_idx, ==);
    freq = selector_width(same_clause);
    quanti_pos = aggregate(same_clause, indicator(is_quantifier));
    clause_QD = quanti_pos*freq;
    QD_type3 = sort(clause_QD, clause_QD)[-1];

    return QD_type3;
}

     console function: QD_type3_fn()


#### Standard-3: Test Cases (4/4)

In [154]:
type3_1 = "∀vG(v)→∃iE(i)→∀o∀zC(z,o)"; # QD=2
type3_2 = "∀r∀j¬N(r,j)∧∀l∃yN(y,l)→∃e∃zN(e,z)"; # QD=2
type3_3 = "∀tZ(t)∧∀y∀vZ(y,v)→∀xH(x)"; # QD=2
type3_4 = "∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)"; # QD=2

    value: type3_1 =  "∀vG(v)→∃iE(i)→∀o∀zC(z,o)"
    value: type3_2 =  "∀r∀j¬N(r,j)∧∀l∃yN(y,l)→∃e∃zN(e,z)"
    value: type3_3 =  "∀tZ(t)∧∀y∀vZ(y,v)→∀xH(x)"
    value: type3_4 =  "∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)"


In [155]:
set example type3_1
QD_type3_1 = QD_type3_fn();
# draw(QD_type3_fn(), "∀vG(v)→∃iE(i)→∀o∀zC(z,o)");

     s-op: QD_type3_1
 	 Example: QD_type3_1("∀vG(v)→∃iE(i)→∀o∀zC(z,o)") = [2.0]*24 (floats)


In [156]:
set example type3_2
QD_type3_2 = QD_type3_fn();
# draw(QD_type3_fn(), "∀r∀j¬N(r,j)∧∀l∃yN(y,l)→∃e∃zN(e,z)");

     s-op: QD_type3_2
 	 Example: QD_type3_2("∀r∀j¬N(r,j)∧∀l∃yN(y,l)→∃e∃zN(e,z)") = [2.0]*33 (floats)


In [157]:
set example type3_3
QD_type3_3 = QD_type3_fn();
# draw(QD_type3_fn(), "∀tZ(t)∧∀y∀vZ(y,v)→∀xH(x)");

     s-op: QD_type3_3
 	 Example: QD_type3_3("∀tZ(t)∧∀y∀vZ(y,v)→∀xH(x)") = [2.0]*24 (floats)


In [171]:
set example type3_4
QD_type3_4 = QD_type3_fn();
# draw(QD_type3_fn(), "∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)");

     s-op: QD_type3_4
 	 Example: QD_type3_4("∀xP(x)∧∀y∀zQ(y,z)→∃wR(w)") = [2.0]*24 (floats)


### **Nested-1**: Nested QD with Type 2
- The deepest embedded elementary expression = Standard-2
- It can also process **Standard-2 QD**.

#### Nested-1: Algorithm

In [160]:
def layer_QD_type4(num_layer){
    # Compute the max QD in different layers
    # num: The number of selected layer

    ## 1. Select the layer and the positions of quantifier and LO
    layer = select(adjusted_depth, num_layer, ==);
    layer_quanti = select(is_quantifier, True, ==) and layer;
    layer_LO = select(is_LO, True, ==) and layer;
    ### Find whether this layer has LO for later "if-else"
    has_LO = aggregate(layer_LO, 1, 0) > 0;
    LO_idx = aggregate(layer_LO, indices);

    ## 2. Extact tokens on both sides of LO and compute their respective quantifier
    left_quant  = layer_quanti and select(indices, LO_idx, <);
    right_quant = layer_quanti and select(indices, LO_idx, >);
    left_QD  = selector_width(left_quant);
    right_QD = selector_width(right_quant);

    ## 3. Consider other conditions
    ### To avoid the empty pre-defined layers get some results,
    ### it labels all existing layers in current formula
    ### and fliter the empty layers. 
    no_layer = aggregate(layer,1,0) == 0;  
    ### Directly aggregate all quantifier numbers If there is no LO
    only_QD = selector_width(layer_quanti);

    ## 4. Assign 0 to the empty layer and compare QD from both sides
    layer_QD = 0 if no_layer
                else
               (max(left_QD, right_QD) if has_LO 
                else
               (only_QD));

    return layer_QD;
}

     console function: layer_QD_type4(num_layer)


In [161]:
def QD_type4_fn(){
    # Decide the number of layer and sum each max layer QD together

    ## Theoretically, RASP can process QD with an arbitrary depth;
    ## In practical, as RASP doesn't support explicit recursion or "for" loop,
    ## this problem can only be solved as the following operations  
    ## by pre-defining the highest maximum sublayer depth and trying all possible depths.

    ## 1. Store results from each layer
    ### The usage frequency of the related function
    ### depends on the maximum depth in all layers.
    ### Here I select 4 sublayer to meet the Maximum depth 4.
    ### It can also be solved by 3 layers, which will be explained in the Type6 part.
    layer0_QD = layer_QD_type4(0);
    layer1_QD = layer_QD_type4(1);
    layer2_QD = layer_QD_type4(2);
    layer3_QD = layer_QD_type4(3);
    ## Can continue to add "layer_QD" to meet deeper examples

    QD_type4 = layer0_QD + layer1_QD + layer2_QD + layer3_QD;
    return QD_type4;
}

     console function: QD_type4_fn()


#### Nested-1: Test Cases (5/5)

In [164]:
type4_1 = "∀xP(x)"; # 1
type4_2 = "∀x(P(x)∨∀yQ(y))"; # 2
type4_3 = "∃x(P(x)∨∀y∃zQ(y,z))"; # 3
type4_4 = "∃x(P(x)→∀y(Q(y)∨∀zR(z)))"; # 3
type4_5 = "∀x(P(x)→∃y(∀zQ(y,z)∨∃u∃vR(u,v)))"; # 4

    value: type4_1 =  "∀xP(x)"
    value: type4_2 =  "∀x(P(x)∨∀yQ(y))"
    value: type4_3 =  "∃x(P(x)∨∀y∃zQ(y,z))"
    value: type4_4 =  "∃x(P(x)→∀y(Q(y)∨∀zR(z)))"
    value: type4_5 =  "∀x(P(x)→∃y(∀zQ(y,z)∨∃u∃vR(u,v)))"


In [165]:
set example type4_1
QD_type4_1 = QD_type4_fn();
# draw(QD_type4_fn(), "∀xP(x)");

     s-op: QD_type4_1
 	 Example: QD_type4_1("∀xP(x)") = [1]*6 (ints)


In [166]:
set example type4_2
QD_type4_2 = QD_type4_fn();
# draw(QD_type4_fn(), "∀x(P(x)∨∀yQ(y))");

     s-op: QD_type4_2
 	 Example: QD_type4_2("∀x(P(x)∨∀yQ(y))") = [2]*15 (ints)


In [167]:
set example type4_3
QD_type4_3 = QD_type4_fn();
# draw(QD_type4_fn(), "∃x(P(x)∨∀y∃zQ(y,z))");

     s-op: QD_type4_3
 	 Example: QD_type4_3("∃x(P(x)∨∀y∃zQ(y,z))") = [3]*19 (ints)


In [170]:
set example type4_4
QD_type4_4 = QD_type4_fn();
# draw(QD_type4_fn(), "∃x(P(x)→∀y(Q(y)∨∀zR(z)))");

     s-op: QD_type4_4
 	 Example: QD_type4_4("∃x(P(x)→∀y(Q(y)∨∀zR(z)))") = [3]*24 (ints)
	 =  [3]*24 (ints)


In [169]:
set example type4_5
QD_type4_5 = QD_type4_fn();
# draw(QD_type4_fn(), "∀x(P(x)→∃y(∀zQ(y,z)∨∃u∃vR(u,v)))");

     s-op: QD_type4_5
 	 Example: QD_type4_5("∀x(P(x)→∃y(∀zQ(y,z)∨∃u∃vR(u,v)))") = [4]*32 (ints)
